#### Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [1]:
import warnings

import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
warnings.simplefilter("ignore")
%matplotlib inline

In [ ]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
warnings.simplefilter("ignore")
%matplotlib inline

In [2]:
data = pd.read_csv("data/assignment2/test.csv")

In [3]:
data.head(n=2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.0,W,12473,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.0,W,15651,417.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
target = data["isFraud"]
data = data.drop(["TransactionID", "isFraud"], axis=1)
print("data.shape = {} rows, {} cols".format(*data.shape))

data.shape = 100001 rows, 392 cols


In [34]:
x_train, x_test = train_test_split(
    data, train_size=0.70, shuffle=False
)
y_train, y_test = train_test_split(
    target, train_size=0.70, shuffle=False
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 70000 rows, 392 cols
x_test.shape = 30001 rows, 392 cols


In [35]:
numerical_features = x_train.select_dtypes(exclude=["object"])
numerical_features = numerical_features.columns.tolist()

x_train = x_train[numerical_features]
x_test = x_test[numerical_features]

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 70000 rows, 378 cols
x_test.shape = 30001 rows, 378 cols


In [36]:
model = xgb.XGBClassifier(random_state=1)
model.fit(x_train, y_train,
          eval_set=[(x_train, y_train), (x_test, y_test)], 
          early_stopping_rounds=50, 
          eval_metric="auc",
          verbose=50)

[0]	validation_0-auc:0.78079	validation_1-auc:0.78428
[50]	validation_0-auc:0.97366	validation_1-auc:0.89962
[81]	validation_0-auc:0.98586	validation_1-auc:0.89557


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [37]:
train_score = roc_auc_score(y_train, model.predict(x_train))
test_score = roc_auc_score(y_test, model.predict(x_test))

print(f"Train-score: {round(train_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.752, Test-score: 0.685


In [38]:
train_score_proba = roc_auc_score(y_train, model.predict_proba(x_train)[:, 1])
test_score_proba = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score_proba, 3)}, Test-score: {round(test_score_proba, 3)}")

Train-score: 0.955, Test-score: 0.9


##### Observations: It seems like predict_proba for roc_auc_score gives us a relatively good estimation if transaction is fraud. But we validated our model only on one set of data. So it might not be as good in predicting the fraud transactions on the other set of data. 

#### Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [47]:
x_train, x_valid = train_test_split(
    data, train_size=0.75, shuffle=False
)
y_train, y_valid = train_test_split(
    target, train_size=0.75, shuffle=False
)

x_train = x_train[numerical_features]
x_valid = x_valid[numerical_features]

x_valid, x_test = train_test_split(
    x_valid, train_size=0.75, shuffle=False
)
y_valid, y_test = train_test_split(
    y_valid, train_size=0.75, shuffle=False
)

print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))
print("x_test.shape = {} rows, {} cols".format(*x_test.shape))

x_train.shape = 75000 rows, 378 cols
x_valid.shape = 18750 rows, 378 cols
x_test.shape = 6251 rows, 378 cols


In [48]:
model = xgb.XGBClassifier(random_state=1)
model.fit(x_train, y_train,
          eval_set=[(x_train, y_train),(x_valid, y_valid), (x_test, y_test)], 
          early_stopping_rounds=50, 
          eval_metric="auc",
          verbose=50)

[0]	validation_0-auc:0.78473	validation_1-auc:0.77029	validation_2-auc:0.80284
[50]	validation_0-auc:0.97074	validation_1-auc:0.90176	validation_2-auc:0.92089
[99]	validation_0-auc:0.98829	validation_1-auc:0.90568	validation_2-auc:0.92460


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [49]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(y_test, model.predict(x_test))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.859, Valid-score: 0.684, Test-score: 0.711


In [50]:
train_score_proba = roc_auc_score(y_train, model.predict_proba(x_train)[:, 1])
valid_score_proba = roc_auc_score(y_valid, model.predict_proba(x_valid)[:, 1])
test_score_proba = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])

print(f"Train-score: {round(train_score_proba, 3)}, Valid-score: {round(valid_score_proba, 3)}, Test-score: {round(test_score_proba, 3)}")

Train-score: 0.988, Valid-score: 0.906, Test-score: 0.925


##### Observation: By adding extra set to validate our model, we increased our test score by 2.5 %